In [8]:
# !pip install genomic_benchmarks
# !pip install omegaconf

In [1]:
# using Kernal PyTorch-2.0.1

# from gpn.data import GenomeMSA #, Tokenizer
# import gpn.model
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import torch
from transformers import AutoModel #, AutoModelForMaskedLM
from tqdm import tqdm

# import pickle
# import re
import os
import csv
import warnings
warnings.filterwarnings('ignore')

from datetime import datetime, timedelta

%run hyena_utility.py

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

/scratch/local/48855325/ipykernel_3108526/1965316763.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/home/sunhuaikuan/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device(type='cuda')

In [2]:

# def Cur_Inference(model, tokenizer, max_length, device, sequence):
def Cur_Inference(model, sequence):  # model, tokenizer, max_length, device, 

    '''
    this selects which backbone to use, and grabs weights/ config from HF
    4 options:
      'hyenadna-tiny-1k-seqlen'   # fine-tune on colab ok
      'hyenadna-small-32k-seqlen'
      'hyenadna-medium-160k-seqlen'  # inference only on colab
      'hyenadna-medium-450k-seqlen'  # inference only on colab
      'hyenadna-large-1m-seqlen'  # inference only on colab
    '''

    # you only need to select which model to use here, we'll do the rest!
    # pretrained_model_name = 'hyenadna-small-32k-seqlen'

    # max_lengths = {
    #     'hyenadna-tiny-1k-seqlen': 1024,
    #     'hyenadna-small-32k-seqlen': 32768,
    #     'hyenadna-medium-160k-seqlen': 160000,
    #     'hyenadna-medium-450k-seqlen': 450000,  # T4 up to here
    #     'hyenadna-large-1m-seqlen': 1_000_000,  # only A100 (paid tier)
    # }


    #### Single embedding example ####

    # create a sample 450k long, prepare
    # sequence = 'ACTG' * int(max_length/4)
    tok_seq = tokenizer(sequence)
    tok_seq = tok_seq["input_ids"]  # grab ids

    # place on device, convert to tensor
    tok_seq = torch.LongTensor(tok_seq).unsqueeze(0)  # unsqueeze for batch dim
    tok_seq = tok_seq.to(device)

    # prep model and forward
    # model.to(device)
    # model.eval()
    with torch.inference_mode():
        embeddings = model(tok_seq)

    # cls_embedding = embeddings.last_hidden_state[:, 0, :]
    # cls_embedding = embeddings[:, 0, :]
    
    mean_embeddings = embeddings.mean(dim=1) # Mean across the sequence length dimension
    mean_embeddings = mean_embeddings.squeeze(0)  # This will change the shape to [256]

    
    # print(embeddings.shape)  # embeddings here!
    # return cls_embedding
    return mean_embeddings


In [3]:
# def Subsequence2Embedding(model, tokenizer, max_length, device, subsequence):
def Subsequence2Embedding(subsequence):
    # embeddings = My_Inference(model, tokenizer, max_length, device, subsequence)
    embeddings = Cur_Inference(model, subsequence)
    # print(embeddings.shape)
    # return embeddings[0,int(max_length/2),:]
    return embeddings  # embeddings[0,0,:]

# max_length

In [4]:
def append_rows_to_csv(csv_Filename, rows):
    with open(csv_Filename, mode='a', newline='') as file:
        writer = csv.writer(file)
        # writer.writerow(rows)
        for row in rows:
            writer.writerow(row)

### Main Process

In [5]:
pretrained_model_name = 'hyenadna-small-32k-seqlen'
pretrained_model_name = 'hyenadna-medium-160k-seqlen'
# pretrained_model_name = 'hyenadna-medium-450k-seqlen'
# pretrained_model_name = 'hyenadna-large-1m-seqlen'
model, tokenizer, max_length =  get_model_tokenizer_maxlen(pretrained_model_name)
model.to(device)
model.eval()

Using device: cuda
Git LFS initialized.


Cloning into 'hyenadna-medium-160k-seqlen'...


Loaded pretrained weights ok!


HyenaDNAModel(
  (backbone): LMBackbone(
    (embeddings): GPT2Embeddings(
      (word_embeddings): Embedding(16, 256)
    )
    (layers): ModuleList(
      (0): Block(
        (mixer): HyenaOperator(
          (dropout): Dropout(p=0.0, inplace=False)
          (in_proj): Linear(in_features=256, out_features=768, bias=True)
          (out_proj): Linear(in_features=256, out_features=256, bias=True)
          (short_filter): Conv1d(768, 768, kernel_size=(3,), stride=(1,), padding=(2,), groups=768)
          (filter_fn): HyenaFilter(
            (dropout): Dropout(p=0.0, inplace=False)
            (pos_emb): PositionalEmbedding()
            (implicit_filter): Sequential(
              (0): Linear(in_features=5, out_features=64, bias=True)
              (1): Sin()
              (2): Linear(in_features=64, out_features=64, bias=True)
              (3): Sin()
              (4): Linear(in_features=64, out_features=64, bias=True)
              (5): Sin()
              (6): Linear(in_features=

## Load dna segment data file 

In [6]:
import pandas as pd

# pathogenecity_type='noncoding'
pathogenecity_type='missense'

df=pd.read_csv('dna_segment_'+pathogenecity_type+'.csv')
df

,sequence,y
0,TCCTGTCCAGTCCCGTCCCCGGCGCGGCCCGCGCGCTCCTCCGCCG...,0
1,CCTGTCCAGTCCCGTCCCCGGCGCGGCCCGCGCGCTCCTCCGCCGC...,0
2,GTCCCGTCCCCGGCGCGGCCCGCGCGCTCCTCCGCCGCCTCTCGCC...,0
3,CGGCGCGGCCCGCGCGCTCCTCCGCCGCCTCTCGCCTGCGCCATGG...,0
4,CGCGGCCCGCGCGCTCCTCCGCCGCCTCTCGCCTGCGCCATGGCCG...,0
...,...,...
560183,ACAGCTTTCAGTGCAAAGGAAGGAAGAGCTTCTCCGGAGAGCGGGA...,0
560184,CAGCTTTCAGTGCAAAGGAAGGAAGAGCTTCTCCGGAGAGCGGGAA...,0
560185,TCCGGAGAGCGGGAATATTCTCTTGCACAGCTGGACTGTAATCATC...,0
560186,CGGAGAGCGGGAATATTCTCTTGCACAGCTGGACTGTAATCATCGC...,0


In [7]:
%%time

now = datetime.now()
formatted_time = now.strftime("%y-%m-%d-%H-%M-%S")
csv_filename = '/home/sunhuaikuan/ondemand/blue_papers/DNA_LLM_REVIEW/preprocess/pathogenecity/pathogenecity_hyena_'+pathogenecity_type+'_'+formatted_time+'.csv'


rows=[]
for index, row in df.iterrows():      
        
    y=row['y']

    subsequence = row['sequence']
    if 'N' in subsequence:
        print("The character 'N' is present in the string.")
        
    embedding = Subsequence2Embedding(subsequence)
    # print(embedding.shape)

    # feature=np.array(embedding_df.iloc[64])
    rows.append(np.append(embedding.cpu().numpy(),  [ y]))


    if index > 0 and (index % 2000) == 0:
        append_rows_to_csv(csv_filename, rows)
        rows=[]
        print (f"index = {index} completed")
        
append_rows_to_csv(csv_filename, rows)

print(f"Create File: "+csv_filename)

index = 2000 completed
index = 4000 completed
index = 6000 completed
index = 8000 completed
index = 10000 completed
index = 12000 completed
index = 14000 completed
index = 16000 completed
index = 18000 completed
index = 20000 completed
index = 22000 completed
index = 24000 completed
index = 26000 completed
index = 28000 completed
index = 30000 completed
index = 32000 completed
index = 34000 completed
index = 36000 completed
index = 38000 completed
index = 40000 completed
index = 42000 completed
index = 44000 completed
index = 46000 completed
index = 48000 completed
index = 50000 completed
index = 52000 completed
index = 54000 completed
index = 56000 completed
index = 58000 completed
index = 60000 completed
index = 62000 completed
index = 64000 completed
index = 66000 completed
index = 68000 completed
index = 70000 completed
index = 72000 completed
index = 74000 completed
index = 76000 completed
index = 78000 completed
index = 80000 completed
index = 82000 completed
index = 84000 comple

### Load CSV File

In [8]:
import pandas as pd

# csv_filename='/home/sunhuaikuan/ondemand/blue_papers/DNA_LLM_REVIEW/preprocess/pathogenecity/pathogenecity_hyena_noncoding_24-10-29-16-31-33.csv'

def load_embedding_file(csv_filename):

    df=pd.read_csv(csv_filename)
    
    column_names = [f'{i}' for i in range(1, df.shape[1])]
    column_names.extend([ 'y'])
    
    df.columns = column_names
    return df
    

df = load_embedding_file(csv_filename)
df

,1,2,3,4,5,6,7,8,9,10,...,248,249,250,251,252,253,254,255,256,y
0,-0.165760,-0.775195,-0.615126,-0.971025,-1.913011,-0.801352,1.159068,-0.685350,-0.653669,-0.096608,...,0.310376,0.456689,-1.007345,-1.606998,-1.618424,1.240193,0.473080,0.256684,-0.935613,0.0
1,-0.137474,-0.788819,-0.607544,-0.950123,-1.989848,-0.770073,1.134804,-0.698152,-0.754873,-0.122203,...,0.384797,0.452388,-1.012480,-1.564671,-1.634364,1.239771,0.449268,0.295268,-0.885379,0.0
2,-0.057533,-0.791069,-0.616168,-0.836501,-1.967007,-0.733224,1.111632,-0.683399,-0.791432,-0.109900,...,0.378847,0.415309,-1.069680,-1.575123,-1.551741,1.206547,0.393325,0.321506,-0.889493,0.0
3,-0.059399,-0.755095,-0.629050,-0.842900,-1.939599,-0.740944,1.113892,-0.703221,-0.720661,-0.143797,...,0.358683,0.422459,-1.065204,-1.585578,-1.554344,1.212077,0.379778,0.301417,-0.937824,0.0
4,-0.071139,-0.729199,-0.606902,-0.829894,-1.892584,-0.728332,1.110167,-0.698343,-0.722920,-0.116437,...,0.378509,0.431281,-1.067796,-1.594991,-1.537138,1.223300,0.392155,0.332627,-0.926249,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560182,-0.148571,-0.792183,-0.521046,-0.929553,-0.910005,-0.921572,1.201978,-0.803206,-0.155598,0.485060,...,0.492746,0.479940,-0.994844,-1.504475,-1.514782,1.257350,0.534343,0.462038,-0.826865,0.0
560183,-0.125961,-0.783654,-0.528770,-0.899592,-0.896188,-0.903221,1.209326,-0.833910,-0.120305,0.475503,...,0.498197,0.465864,-1.018678,-1.494310,-1.495789,1.259540,0.528486,0.434672,-0.848990,0.0
560184,-0.030807,-0.759196,-0.487905,-0.758114,-0.790885,-0.849533,1.267575,-0.909575,-0.073220,0.375187,...,0.557983,0.488931,-0.927740,-1.476444,-1.447461,1.198302,0.481330,0.553817,-0.920865,0.0
560185,-0.010619,-0.762725,-0.473467,-0.705690,-0.751864,-0.809319,1.249628,-0.890070,-0.093685,0.397889,...,0.617771,0.488129,-0.951747,-1.440598,-1.383776,1.210845,0.476213,0.580937,-0.887212,0.0
